# HW1 DSAI HW1 Peak Load Forecasting
* [作業說明](https://hackmd.io/APjYJ2KtQ6CPI0MMUyjmjA)

## 執行
* 安裝套件
`$ pip install -r requirements.txt`

* 執行程式
`$ python3 app.py`

## 開發紀錄
* 使用的 dataset
    * [台灣電力公司_過去電力供需資訊](https://data.gov.tw/dataset/19995)
    * [台灣電力公司_未來一週電力供需預測](https://data.gov.tw/dataset/33462)

使用到 `日期`, `淨尖峰供電能力(MW)`, `備轉容量(MW)`, `備轉容量率(%)`

`尖峰負載(MW)` 則是用來比較正確性的

## 以下為 code 解釋 ( 會稍微修改 code )

* 讀去資料時發現有錯誤的格式，所以寫了一小段自動去除的 code

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
# read data set
data = pd.read_csv("data/2017.1.1 ~ 2019.1.31 的電力供需.csv", encoding='utf-8')
data = data[["日期","淨尖峰供電能力(MW)","尖峰負載(MW)","備轉容量(MW)","備轉容量率(%)"]]

# remove error data
data = data.reset_index()

print(np.isnan(data).any())
print(data.shape)
print("上面結果為 true 則表示有資料是錯的\n")

data.dropna(inplace=True)
print("處理後")
print(np.isnan(data).any())
print(data.shape)

index          False
日期             False
淨尖峰供電能力(MW)    False
尖峰負載(MW)        True
備轉容量(MW)       False
備轉容量率(%)       False
dtype: bool
(760, 6)
上面結果為 true 則表示有資料是錯的

處理後
index          False
日期             False
淨尖峰供電能力(MW)    False
尖峰負載(MW)       False
備轉容量(MW)       False
備轉容量率(%)       False
dtype: bool
(759, 6)


* 註解掉的部份為將 dataset 一部分用作 test 來測試準確率，
現在是將這個部份都拿來 train module

In [2]:
X = data[["日期","淨尖峰供電能力(MW)","備轉容量(MW)","備轉容量率(%)"]]
y = data[["尖峰負載(MW)"]]


# 75% 劃分為 train set, 25%劃分為 test set
# X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1)

# train module
linreg = LinearRegression()
linreg.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

* 接著是使用台電的預測資料，修改成我要的格式

In [3]:
# read support data
pre_data = pd.read_csv("data/power_supply_predicted.csv", encoding='utf-8')
pre_data = pre_data.rename(index=str,columns={"日期(年/月/日)":"日期","預估淨尖峰供電能力(萬瓩)":"淨尖峰供電能力(MW)","預估瞬時尖峰負載(萬瓩)":"尖峰負載(MW)","預估尖峰備轉容量率(%)":"備轉容量率(%)","預估尖峰備轉容量(萬瓩)":"備轉容量(MW)"})
# modify format
for i in range(0, 7):
    s=pre_data.at[str(i), "日期"]
    pre_data.at[str(i), "日期"]=s[0:4]+s[5:7]+s[8:10]
    pre_data.at[str(i), "淨尖峰供電能力(MW)"]*=10
    pre_data.at[str(i), "備轉容量(MW)"]*=10
pre_data = pre_data[["日期","淨尖峰供電能力(MW)","備轉容量(MW)","備轉容量率(%)"]]    
print(pre_data)


         日期  淨尖峰供電能力(MW)  備轉容量(MW)  備轉容量率(%)
0  20190402        30560      1860      6.48
1  20190403        30560      1960      6.85
2  20190404        28140      2440      9.48
3  20190405        27060      2460      9.98
4  20190406        26970      2670     10.98
5  20190407        26930      2430      9.90
6  20190408        30600      2100      7.37


* 用剛剛產生的 module 去測試，但是實際上`尖峰負載`的值會與`淨尖峰供電能力`、`備轉容量`有非常大的相關，所以基本上會和台電的預測結果相差不遠

In [4]:
# predict future peak load for a week and output
pred = linreg.predict(pre_data)
pred_output = pd.DataFrame(pred, columns=["peak_load(MW)"])
pred_output.insert(0,"date", pre_data["日期"].to_numpy())
print(pred_output)
pred_output.to_csv("submission.csv", encoding='utf-8', index=False)

       date  peak_load(MW)
0  20190402        28700.0
1  20190403        28600.0
2  20190404        25700.0
3  20190405        24600.0
4  20190406        24300.0
5  20190407        24500.0
6  20190408        28500.0
